In [1]:
from flask import Flask, request
import requests
import json

app = Flask(__name__)

class TrendyolEntegrasyonu:
    def __init__(self, api_key, supplier_id):
        self.api_key = api_key
        self.supplier_id = supplier_id
        
    def iade_taleplerini_al(self):
        url = f"https://api.trendyol.com/sapigw/suppliers/{self.supplier_id}/v2/returns"

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Basic {self.api_key}"
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            iade_talepleri = response.json()
            return iade_talepleri
        else:
            print(f"Iade talepleri alınamadı")
            print(f"HTTP Hata Kodu: {response.status_code}")
            print(f"API Yanıtı: {response.text}")
            return None
    
    def iade_talebini_onayla(self, iade_id):
        url = f"https://api.trendyol.com/sapigw/suppliers/{self.supplier_id}/v2/returns/{iade_id}/approve"

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Basic {self.api_key}"
        }

        response = requests.put(url, headers=headers)

        if response.status_code == 200:
            print(f"Iade talebi onaylandı: İade ID - {iade_id}")
        else:
            print(f"Iade talebi onaylanamadı: İade ID - {iade_id}")
            print(f"HTTP Hata Kodu: {response.status_code}")
            print(f"API Yanıtı: {response.text}")
    
    def iade_talebini_reddet(self, iade_id):
        url = f"https://api.trendyol.com/sapigw/suppliers/{self.supplier_id}/v2/returns/{iade_id}/reject"

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Basic {self.api_key}"
        }

        response = requests.put(url, headers=headers)

        if response.status_code == 200:
            print(f"Iade talebi reddedildi: İade ID - {iade_id}")
        else:
            print(f"Iade talebi reddedilemedi: İade ID - {iade_id}")
            print(f"HTTP Hata Kodu: {response.status_code}")
            print(f"API Yanıtı: {response.text}")
    
    def iade_takibini_sagla(self, iade_id):
        url = f"https://api.trendyol.com/sapigw/suppliers/{self.supplier_id}/v2/returns/{iade_id}"

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Basic {self.api_key}"
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            iade_bilgileri = response.json()
            print(f"Iade takibi: İade ID - {iade_id}")
            print("Detaylar:")
            print(iade_bilgileri)
            
        else:
            print(f"Iade takibi yapılamadı: İade ID - {iade_id}")
            print(f"HTTP Hata Kodu: {response.status_code}")
            print(f"API Yanıtı: {response.text}")
    
    
    def kargo_takip(self, tracking_number):
        url = f"https://api.trendyol.com/sapigw/suppliers/{self.supplier_id}/v2/shipments/{tracking_number}"

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Basic {self.api_key}"
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            shipment_data = response.json()
            print(f"Kargo takibi: Takip Numarası - {tracking_number}")
            print("Detaylar:")
            print(shipment_data)
        else:
            print(f"Kargo takibi yapılamadı: Takip Numarası - {tracking_number}")
            print(f"HTTP Hata Kodu: {response.status_code}")
            print(f"API Yanıtı: {response.text}")

    def webhook_istegini_isle(self, veri):
        if veri["eventType"] == "OrderCreated":
            
            self.stok_guncelle(veri)
            self.fatura_olustur(veri)
            self.iade_taleplerini_al(veri)
            self.iade_talebini_onayla(veri)
            self.iade_talebini_reddet(veri)
            self.kargo_takip(veri)
            
            
        # Diğer işlemler ve koşullar

    def stok_guncelle(self, veri):
        siparis_id = veri["data"]["orderId"]
        urunler = veri["data"]["orderItems"]

        for urun in urunler:
            barkod = urun["barcode"]
            miktar_degisimi = -1  # Stok düşürme için -1 değerini kullanabilirsiniz
            self.stok_guncelleme_istegi_gonder(barkod, miktar_degisimi, siparis_id)
    
    def stok_guncelleme_istegi_gonder(self, barkod, miktar_degisimi, siparis_id,veri):
        url = f"https://api.trendyol.com/sapigw/suppliers/{self.supplier_id}/v2/products/{barkod}/stock"

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Basic {self.api_key}"
        }

        payload = {
            "quantity": miktar_degisimi
        }

        yanit = requests.put(url, headers=headers, data=json.dumps(payload))

        if yanit.status_code == 200:
            print(f"Stok güncelleme başarılı: Sipariş ID - {siparis_id}")
        else:
            print(f"Stok güncelleme hatası: Sipariş ID - {siparis_id}")
            print(f"HTTP Hata Kodu: {yanit.status_code}")
            print(f"API Yanıtı: {yanit.text}")
            
    def stok_artir(self, barkod, miktar_artisi,veri):
        url = f"https://api.trendyol.com/sapigw/suppliers/{self.supplier_id}/v2/products/{barkod}/stock"

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Basic {self.api_key}"
        }

        payload = {
            "quantity": miktar_artisi
        }

        yanit = requests.put(url, headers=headers, data=json.dumps(payload))

        if yanit.status_code == 200:
            print(f"Stok artırma başarılı: Barkod - {barkod}")
        else:
            print(f"Stok artırma hatası: Barkod - {barkod}")
            print(f"HTTP Hata Kodu: {yanit.status_code}")
            print(f"API Yanıtı: {yanit.text}")

    def fatura_olustur(self, veri):
        siparis_id = veri["data"]["orderId"]
        fatura_id = veri["data"]["invoiceId"]
        fatura_verisi = self.fatura_bilgilerini_al(fatura_id)

        if fatura_verisi:
            self.fatura_olusturma_istegi_gonder(fatura_verisi, siparis_id)

    def fatura_bilgilerini_al(self, fatura_id,veri):
        url = f"https://api.trendyol.com/sapigw/invoicing/{fatura_id}"

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Basic {self.api_key}"
        }

        yanit = requests.get(url, headers=headers)

        if yanit.status_code == 200:
            fatura_verisi = yanit.json()
            return fatura_verisi
        else:
            print(f"Fatura bilgisi alınamadı: Fatura ID - {fatura_id}")
            print(f"HTTP Hata Kodu: {yanit.status_code}")
            print(f"API Yanıtı: {yanit.text}")
            return None

    def fatura_olusturma_istegi_gonder(self, fatura_verisi, siparis_id,veri):
        url = f"https://api.trendyol.com/sapigw/suppliers/{self.supplier_id}/v2/invoices"
        
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Basic {self.api_key}"
        }

        payload = {
            "invoiceData": fatura_verisi
        }

        yanit = requests.post(url, headers=headers, data=json.dumps(payload))

        if yanit.status_code == 200:
            print(f"Fatura oluşturma başarılı: Sipariş ID - {siparis_id}")
        else:
            print(f"Fatura oluşturma hatası: Sipariş ID - {siparis_id}")
            print(f"HTTP Hata Kodu: {yanit.status_code}")
            print(f"API Yanıtı: {yanit.text}")

    def fiyat_guncelle(self, barkod, yeni_fiyat):
        url = f"https://api.trendyol.com/sapigw/suppliers/{self.supplier_id}/v2/products/{barkod}/prices"

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Basic {self.api_key}"
        }

        payload = {
            "price": yeni_fiyat
        }

        yanit = requests.put(url, headers=headers, data=json.dumps(payload))

        if yanit.status_code == 200:
            print(f"Fiyat güncelleme başarılı: Barkod - {barkod}")
        else:
            print(f"Fiyat güncelleme hatası: Barkod - {barkod}")
            print(f"HTTP Hata Kodu: {yanit.status_code}")
            print(f"API Yanıtı: {yanit.text}")
            
            
    def yeni_urun_ekle(self, urun_bilgileri):
        url = f"https://api.trendyol.com/sapigw/suppliers/{self.supplier_id}/v2/products"

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Basic {self.api_key}"
        }

        payload = {
            "product": urun_bilgileri
        }

        response = requests.post(url, headers=headers, data=json.dumps(payload))

        if response.status_code == 200:
            print("Yeni ürün ekleme başarılı")
            print(f"Ürün bilgileri: {urun_bilgileri}")
        else:
            print("Yeni ürün eklenemedi")
            print(f"HTTP Hata Kodu: {response.status_code}")
            print(f"API Yanıtı: {response.text}")

    def urun_guncelle(self, urun_bilgileri):
        barkod = urun_bilgileri["barcode"]
        url = f"https://api.trendyol.com/sapigw/suppliers/{self.supplier_id}/v2/products/{barkod}"

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Basic {self.api_key}"
        }

        payload = {
            "product": urun_bilgileri
        }

        response = requests.put(url, headers=headers, data=json.dumps(payload))

        if response.status_code == 200:
            print(f"Ürün güncelleme başarılı: Barkod - {barkod}")
            print(f"Güncellenen bilgiler: {urun_bilgileri}")
        else:
            print(f"Ürün güncellenemedi: Barkod - {barkod}")
            print(f"HTTP Hata Kodu: {response.status_code}")
            print(f"API Yanıtı: {response.text}")

    def urun_stok_kontrolu(self, barkod,veri):
        url = f"https://api.trendyol.com/sapigw/suppliers/{self.supplier_id}/v2/products/{barkod}/stock"

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Basic {self.api_key}"
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            stok_bilgisi = response.json()
            print(f"Ürün stok kontrolü: Barkod - {barkod}")
            print("Stok Bilgisi:")
            print(stok_bilgisi)
        else:
            print(f"Ürün stok kontrolü yapılamadı: Barkod - {barkod}")
            print(f"HTTP Hata Kodu: {response.status_code}")
            print(f"API Yanıtı: {response.text}")
            
            